In [183]:
!pip install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=533bf90311c680178ecf9d961274dcff33313a8855b8517b2c2e79fd8932bf7a
  Stored in directory: /Users/jeongda-eun/Library/Caches/pip/wheels/a0/b8/b7/8c942b2c5be5158b874a88195116b05ad124bac795f6665e65
Successfully built fake-useragent


In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
import json
import requests
from selenium.webdriver.common.action_chains import ActionChains



In [2]:
def scroll_down():
    pre_height = driver.execute_script("return document.body.scrollHeight") # 현재 스크롤 위치 저장
    try_num = 0
    while True :
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # 스크롤을 가장 아래로 내린다
        time.sleep(1)
        cur_height = driver.execute_script("return document.body.scrollHeight")  # 현재 스크롤을 저장한다.
        try_num += 1
        # print(try_num)
        if pre_height == cur_height :
            break
        else:
            pre_height = cur_height
    driver.implicitly_wait(100)
    
    

def go_page(region):
    # url입력
    url = "https://www.yogiyo.co.kr/mobile/#" # 사이트 입력
    driver.get(url) # 사이트 오픈
    driver.maximize_window() # 전체장
    driver.implicitly_wait(1) # 2초 지연

    # 검색창 선택
    xpath = '''//*[@id="search"]/div/form/input'''  # 검색창
    element = driver.find_element_by_xpath(xpath)
    element.clear()
    driver.implicitly_wait(1)

    # 검색창 입력

    # value = input("지역을 입력하세요")
    value = region
    element.send_keys(value)
    driver.implicitly_wait(1)

    # 검색버튼 클릭
    search_xpath = '''//*[@id="button_search_address"]/button[2]'''
    search = driver.find_element_by_xpath(search_xpath)
    driver.execute_script("arguments[0].click();", search)


    # 검색 콤보상자 선택
    search_result_selector = '#search > div > form > ul > li:nth-child(3) > a'
    search_result = driver.find_element_by_css_selector(search_result_selector)
    driver.execute_script("arguments[0].click();", search_result)
    driver.implicitly_wait(3)


def get_info():
    errors = 0
    # 페이지 소스 출력
    html = driver.page_source
    html_source = BeautifulSoup(html, 'html.parser')
    # print(html_source)
    store_list = []
    stores = driver.find_elements_by_class_name('col-sm-6')
    stores_num = len(list(set(html_source.find_all('div', class_='restaurant-name ng-binding'))))
    for s in stores:
        store_list.append(s)
        # print(s.text)
    # print(len(store_list))
    stores_num = len(list(set(stores)))
    stores_num = list(range(stores_num))
    print('stores_num : ', stores_num)
    scroll_down()
    return stores_num

def goto_store(num):
    cnt_error = 0
    try:
        scroll_down()
        #상점 이동
        in_store_xpath = f'''//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]'''
        in_store = driver.find_element_by_xpath(in_store_xpath)
        time.sleep(2)
        store_name = in_store.text
        print('store_name :', store_name)
        # in_store.click()
        driver.execute_script("arguments[0].click();", in_store)
        # ActionChains(driver).double_click(in_store)
        driver.implicitly_wait(3)
        #리뷰 탭 들어가기
        review_btn_xpath = '//*[@id="content"]/div[2]/div[1]/ul/li[2]/a'
        review_btn = driver.find_element_by_xpath(review_btn_xpath)
        driver.execute_script("arguments[0].click();", review_btn)
        driver.implicitly_wait(3)
        print(f'{store_name} get review btn')
        #리뷰 더보기
        i = 0
        review_errors = 0
        while True:
            i += 1
            var = i*10 + 2
            plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'
            # print(plus_btn_xpath)
            try:
                plus_btn = driver.find_element_by_xpath(plus_btn_xpath)
                # plus_btn.click()
                driver.execute_script("arguments[0].click();", plus_btn)
                print(i)
            except Exception as error:
                review_error_string = str(error)
                print(review_error_string)
                review_errors += 1
            if review_errors > 2:
                break
    except Exception as error:
        error_string = str(error)
        print(error_string)
    return store_name
        
def get_reviews(store_name):
    store_name = store_name
    html = driver.page_source
    html_source = BeautifulSoup(html, 'html.parser')
    reviews = html_source.find_all('p', attrs={'ng-show':'review.comment',
                                              'ng-bind-html':'review.comment|strip_html'})
    reviews = [r.text for r in reviews]
    taste_star = html_source.find_all('span', attrs={'class':'points ng-binding',
                                                       'ng-show':"review.rating_taste > 0"})
    taste_star = [t.text for t in taste_star]
    quantity_star = html_source.find_all('span', attrs={'class':'points ng-binding',
                                                       'ng-show':"review.rating_quantity > 0"})
    quantity_star = [q.text for q in quantity_star]
    delivery_star = html_source.find_all('span', attrs={'class':'points ng-binding',
                                                       'ng-show':"review.rating_delivery > 0"})
    delivery_star = [d.text for d in delivery_star]
    stars = {'taste':taste_star, 'quantity':quantity_star, 'delivery':delivery_star}
    driver.implicitly_wait(2)
      
    return reviews, stars
    # return store_list, review_list, star_list, raw_list

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
options.add_argument("--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 6_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/6.0 Mobile/10A5376e Safari/8536.25")
options.add_argument("--start-maximized")

# headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36",}
# region = '횡성읍'

# go_page(region)
# stores_num = get_info()
# print(stores_num)
# get_reviews(stores_num)

def before_get_review(region):
    go_page(region)
    store_num = get_info()
    return store_num


def get_total_data(num): 
    driver.implicitly_wait(2)
    before_get_review(region)
    store_name=goto_store(num)
    print(f"{num+1} go to store completed")
    reviews, stars = get_reviews(store_name)
    driver.back()
    driver.implicitly_wait(3)
    print(num+1, '뒤로가기 완료')
    print(f'finish {store_name} job completed') 
    return store_name, reviews, stars




 

In [4]:
from tqdm import tqdm
from test import get_total_data, before_get_review
from multiprocessing import Pool
import multiprocessing as multi
import chromedriver_autoinstaller

chromedriver_autoinstaller.install()  # 자동으로 chromedriver 설치

driver = webdriver.Chrome()

region = '한성대학교'
store_num = before_get_review(region)
driver.close()
process_cnt = 4

# result = []
# result.append(get_total_data([region, 1]))
with Pool(process_cnt) as pool:
    imap = pool.map(get_total_data, [[region, i] for i in store_num])
    result = list(tqdm(imap, total=len(store_num), desc='processing'))
print("--------------------------------")
print(result)

위생인증여부 False
store_name : 요리왕-성북구청점
요리왕-성북구청점 get review btn
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[232]/a"}
  (Session info: chrome=119.0.6045.200)
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[242]/a"}
  (Session info: chrome=119.0.6045.200)
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[252]/a"}
  (Session info: chrome=119.0.6045.200)

2 go to store completed
배달금액 : -1, 최소주문금액 : 10000
2 뒤로가기 완료
finish 요리왕-성북구청점 job completed
--------------------------------
[('요리왕-성북구청점', ['잡채밥은 첨시켰는데 맛있어요!', '맛과 양까지 다 최고예요', '맛있게 잘먹었습니다', '맛있게 잘 먹었습니다.', '아니 제가 도착하면 문자해달라고 체크해놧는데\n한시간 넘게 지나서도 안오길레 문자가 혹시나 해서\n문밖에 보니깐 음식 와있네요^^\n이미 음식은 추워서 다 식엇고 차가운상태네요\n문자하나 써주는게 뭐가 어렵다고 참나\n다신 안시켜먹을듯 하네요', '너무 맛있어

In [6]:
len(list(result[0][2].values())[2])
# result[0][2]

222

In [9]:
list(result[0][2].values())

[['5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '4',
  '4',
  '4',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '4',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '3',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '5',
  '4',
  '5',
  '5',

In [ ]:
len(result[0][1])

In [11]:
import pandas as pd
from tqdm import tqdm

df = pd.DataFrame()
for i in tqdm(range(len(result))):
    # list로 packing된 값 unpacking
    name, reviews, stars, clean, delivery_cost, least_cost, reviewEvent = result[i]
    star_key = list(stars.keys())
    # print(stars)
    for idx in tqdm(range(len(reviews))):
        try:
            rev = reviews[idx]
            taste_star = stars[star_key[0]][idx]
            quantity_star = stars[star_key[1]][idx]
            delivery_star = stars[star_key[2]][idx]
            
            
            # concat 함수를 사용하여 DataFrame에 행 추가
            df = pd.concat([df, pd.DataFrame({'name': [name], 
                                               'cleanMark': [clean],
                                               'delivery_cost': [delivery_cost],
                                               'least_cost': [least_cost],
                                               'reviewEvent': [reviewEvent],
                                               'taste_star': [taste_star],
                                               'quantity_star': [quantity_star],
                                               'delivery_star': [delivery_star],
                                               'reviews': [rev]})], ignore_index=True)
            # df = df.append({'name':name, 
            #                 'cleanMark':clean,
            #                 'delivery_cost':delivery_cost,
            #                 'least_cost':least_cost,
            #                 'reviewEvent':reviewEvent,
            #                 'taste_star':taste_star,
            #                 'quantity_star':quantity_star,
            #                 'delivery_star':delivery_star,
            #                 'reviews':rev}, ignore_index=True)
        except:
            pass
print(df)

100%|██████████| 1/1 [00:00<00:00,  6.48it/s]

          name  cleanMark  delivery_cost  least_cost  reviewEvent taste_star  \
0    요리왕-성북구청점       True             -1       10000        False          5   
1    요리왕-성북구청점       True             -1       10000        False          5   
2    요리왕-성북구청점       True             -1       10000        False          5   
3    요리왕-성북구청점       True             -1       10000        False          5   
4    요리왕-성북구청점       True             -1       10000        False          1   
..         ...        ...            ...         ...          ...        ...   
217  요리왕-성북구청점       True             -1       10000        False          5   
218  요리왕-성북구청점       True             -1       10000        False          5   
219  요리왕-성북구청점       True             -1       10000        False          5   
220  요리왕-성북구청점       True             -1       10000        False          4   
221  요리왕-성북구청점       True             -1       10000        False          5   

    quantity_star delivery_star  \
0   

In [12]:
df['reviews'][10]

'배달도 빠르고 가격도 착하고 맛도 굿'

In [13]:
def cleanhtml(raw_html):
    cleanr = re.compile('\n')
    # cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [14]:
df['reviews'] = df['reviews'].apply(lambda x:cleanhtml(x))

In [15]:
df['reviews'][10]

'배달도 빠르고 가격도 착하고 맛도 굿'

In [16]:
df.to_csv('./reviews.csv', encoding='utf-8=sig')